# QA&QC and Data Processing of Marine Magnetics Data

This notebook is built for QA&QC and data processsing of marine magnetics.

## Importing the data

In [1]:
import pandas as pd

## Change here for your local data directory
data_dir = 'marco-mag/data/Magnetometry/merged_data'

df_raw = pd.read_csv(data_dir + '/can_mag_raw.csv', low_memory=False)
df_raw



,Reading_Date,Reading_Time,Magnetic_Field,Signal_Quality,Depth,Leak,Measurement_Time,Signal_Strength,Gradient_Condition,Weak_Signal,...,source_file,__survey_datetime_utc,DATE_vss,TIME_vss,DOY_vss,VSSX_vss,VSSY_vss,VSSZ_vss,VSSF_vss,Profile_Dist_m
0,20-Aug-2025,17:36:36.700,22759.127,79,0.0,0,215,119,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:36.700000+00:00,NaN,NaN,232.0,15878.953200,-6747.600383,-15509.501750,23199.548250,0.000000e+00
1,20-Aug-2025,17:36:37.200,22757.742,79,0.0,0,215,122,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:37.200000+00:00,NaN,NaN,232.0,15878.951200,-6747.597800,-15509.503000,23199.547000,5.950546e+05
2,20-Aug-2025,17:36:37.700,22756.970,89,0.0,0,215,129,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:37.700000+00:00,NaN,NaN,232.0,15878.949200,-6747.595217,-15509.504250,23199.545750,5.950554e+05
3,20-Aug-2025,17:36:38.700,22753.489,89,0.0,0,215,133,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:38.700000+00:00,NaN,NaN,232.0,15878.945200,-6747.590050,-15509.506750,23199.543250,5.950569e+05
4,20-Aug-2025,17:36:39.200,22750.916,89,0.0,0,215,134,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:39.200000+00:00,NaN,NaN,232.0,15878.943200,-6747.587467,-15509.508000,23199.542000,5.950577e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78413,21-Aug-2025,18:00:44.700,22585.304,89,0.0,0,215,132,NaN,NaN,...,can_250821_S6_B..csv,2025-08-21 18:00:44.700000+00:00,NaN,NaN,233.0,15877.485800,-6752.513550,-15522.038600,23208.363400,9.790242e+06
78414,21-Aug-2025,18:00:45.200,22585.829,89,0.0,0,215,129,NaN,NaN,...,can_250821_S6_B..csv,2025-08-21 18:00:45.200000+00:00,NaN,NaN,233.0,15877.476133,-6752.511800,-15522.040933,23208.357733,9.790243e+06
78415,21-Aug-2025,18:00:45.700,22585.503,89,0.0,0,215,133,NaN,NaN,...,can_250821_S6_B..csv,2025-08-21 18:00:45.700000+00:00,NaN,NaN,233.0,15877.466467,-6752.510050,-15522.043267,23208.352067,9.790243e+06
78416,21-Aug-2025,18:00:46.200,22585.352,89,0.0,0,215,133,NaN,NaN,...,can_250821_S6_B..csv,2025-08-21 18:00:46.200000+00:00,NaN,NaN,233.0,15877.456800,-6752.508300,-15522.045600,23208.346400,9.790244e+06


In [2]:
min_lon = df_raw.Mag_Longitude.min()
max_lon = df_raw.Mag_Longitude.max()


print(f"Longitude Mínima: {min_lon}")
print(f"Longitude Máxima: {max_lon}")
print(f"Largura da longitude (em graus): {max_lon - min_lon}")

Longitude Mínima: -48.02843653
Longitude Máxima: -47.90851044
Largura da longitude (em graus): 0.11992608999999987


## QA&QC

QA&QC is conducting through the following steps:

1.   Exclusion of data points such as "Magnetic_Field" < 80 nT;
2.   De-spiking filtering in the time-series of the raw data;
3.   Diurnal correction using the geomagnetic data from the reference observatory;
4.   IGRF removal

### Step #1: Remove data such that 'Signal_Quality' <= 80 nT

In [3]:
 #1. Exclusion of data points such as "Signal_Quality" <= 80
df_filtered = df_raw[df_raw['Signal_Quality'] > 80].copy()
df_filtered = df_raw.copy() # Use this, if you don't want to exlude the bad data points

df_filtered

,Reading_Date,Reading_Time,Magnetic_Field,Signal_Quality,Depth,Leak,Measurement_Time,Signal_Strength,Gradient_Condition,Weak_Signal,...,source_file,__survey_datetime_utc,DATE_vss,TIME_vss,DOY_vss,VSSX_vss,VSSY_vss,VSSZ_vss,VSSF_vss,Profile_Dist_m
0,20-Aug-2025,17:36:36.700,22759.127,79,0.0,0,215,119,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:36.700000+00:00,NaN,NaN,232.0,15878.953200,-6747.600383,-15509.501750,23199.548250,0.000000e+00
1,20-Aug-2025,17:36:37.200,22757.742,79,0.0,0,215,122,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:37.200000+00:00,NaN,NaN,232.0,15878.951200,-6747.597800,-15509.503000,23199.547000,5.950546e+05
2,20-Aug-2025,17:36:37.700,22756.970,89,0.0,0,215,129,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:37.700000+00:00,NaN,NaN,232.0,15878.949200,-6747.595217,-15509.504250,23199.545750,5.950554e+05
3,20-Aug-2025,17:36:38.700,22753.489,89,0.0,0,215,133,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:38.700000+00:00,NaN,NaN,232.0,15878.945200,-6747.590050,-15509.506750,23199.543250,5.950569e+05
4,20-Aug-2025,17:36:39.200,22750.916,89,0.0,0,215,134,NaN,NaN,...,CAN_250820_S6_C.csv,2025-08-20 17:36:39.200000+00:00,NaN,NaN,232.0,15878.943200,-6747.587467,-15509.508000,23199.542000,5.950577e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78413,21-Aug-2025,18:00:44.700,22585.304,89,0.0,0,215,132,NaN,NaN,...,can_250821_S6_B..csv,2025-08-21 18:00:44.700000+00:00,NaN,NaN,233.0,15877.485800,-6752.513550,-15522.038600,23208.363400,9.790242e+06
78414,21-Aug-2025,18:00:45.200,22585.829,89,0.0,0,215,129,NaN,NaN,...,can_250821_S6_B..csv,2025-08-21 18:00:45.200000+00:00,NaN,NaN,233.0,15877.476133,-6752.511800,-15522.040933,23208.357733,9.790243e+06
78415,21-Aug-2025,18:00:45.700,22585.503,89,0.0,0,215,133,NaN,NaN,...,can_250821_S6_B..csv,2025-08-21 18:00:45.700000+00:00,NaN,NaN,233.0,15877.466467,-6752.510050,-15522.043267,23208.352067,9.790243e+06
78416,21-Aug-2025,18:00:46.200,22585.352,89,0.0,0,215,133,NaN,NaN,...,can_250821_S6_B..csv,2025-08-21 18:00:46.200000+00:00,NaN,NaN,233.0,15877.456800,-6752.508300,-15522.045600,23208.346400,9.790244e+06


Let's compare the time-series plots, we are going to use these functions:

In [4]:
# --- prerequisites (safe no-op outside Colab)
def _enable_widgets_for_colab():
    """
    Enable ipywidgets support in Google Colab.

    This helper ensures that interactive widgets (from `ipywidgets`)
    render correctly when running inside Google Colab. It uses
    the Colab `output.enable_custom_widget_manager()` method.
    Safe to call in any environment—does nothing if not in Colab.
    """
    try:
        from google.colab import output  # noqa: F401
        output.enable_custom_widget_manager()
    except Exception:
        pass
_enable_widgets_for_colab()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, HBox, VBox
from IPython.display import display
import matplotlib.ticker as mticker


# ---------- helpers ----------
def _ensure_datetime(df, date_col="Reading_Date", time_col="Reading_Time"):
    """
    Build a UTC-aware datetime Series for the survey data.

    If '__survey_datetime_utc' exists, that is used; otherwise parses
    date_col + time_col (format '%d-%b-%Y %H:%M:%S.%f').
    """
    if "__survey_datetime_utc" in df.columns:
        dt = pd.to_datetime(df["__survey_datetime_utc"], errors="coerce", utc=True)
    else:
        dt = pd.to_datetime(
            df[date_col].astype(str) + " " + df[time_col].astype(str),
            format="%d-%b-%Y %H:%M:%S.%f",
            errors="coerce",
            utc=True,
        )
    return dt


def _to_num(s):
    """Numeric coercion with errors='coerce'."""
    return pd.to_numeric(s, errors="coerce")


def _ordered_unique(seq):
    """Unique in first-appearance order."""
    seen, out = set(), []
    for x in seq:
        x = "" if x is None else str(x)
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out


def _guess_geomag_F_col(df):
    """
    Guess a geomag total-field column (observatory) if present.
    """
    for name in ("VSSF_vss", "F_vss"):
        if name in df.columns:
            return name
    for c in df.columns:
        lc = c.lower()
        if lc.endswith("f_vss") or lc.endswith("vssf_vss"):
            return c
    for c in df.columns:
        if c.endswith("_vss"):
            return c
    return None


def _line_dates_label(d_ts):
    """Compact UTC date label for a line subset that contains '__dt'."""
    if d_ts.empty:
        return "no date"
    dates = d_ts["__dt"].dt.tz_convert("UTC").dt.date.unique()
    dates = np.sort(dates.astype("datetime64[D]")).astype(object)
    if len(dates) == 1:
        return str(dates[0])
    return f"{dates[0]} … {dates[-1]}"


def _profile_distance_from_xy(dsub, x_col="Mag_Easting", y_col="Mag_Northing"):
    """
    Compute along-track cumulative distance (meters) for a single line subset.

    Returns a numpy array S (same length as dsub, in current row order),
    with the first finite point set to 0, NaN where coords are invalid.
    """
    x = _to_num(dsub[x_col]).to_numpy()
    y = _to_num(dsub[y_col]).to_numpy()

    dx = np.full_like(x, np.nan, dtype=float)
    dy = np.full_like(y, np.nan, dtype=float)

    m = np.isfinite(x) & np.isfinite(y)
    if not m.any():
        return np.full(len(dsub), np.nan, dtype=float)

    m_prev = m & np.roll(m, 1)
    dx[m_prev] = x[m_prev] - np.roll(x, 1)[m_prev]
    dy[m_prev] = y[m_prev] - np.roll(y, 1)[m_prev]

    ds = np.hypot(dx, dy)
    inc = np.where(np.isfinite(ds), ds, 0.0)
    S = np.cumsum(inc)
    # keep NaN where coords invalid
    S[~m] = np.nan
    # force first finite to 0
    first_valid = np.argmax(m)
    if m[first_valid]:
        S[first_valid] = 0.0
    return S


# ---------- optional overview map ----------
def plot_lines_overview(df, line_col="line", lat_col="Mag_Latitude", lon_col="Mag_Longitude", s=4):
    """
    Static map overview of lines (scatter by lon/lat).
    """
    d = df.copy()
    d[lon_col] = _to_num(d[lon_col])
    d[lat_col] = _to_num(d[lat_col])
    d = d.dropna(subset=[lon_col, lat_col])
    d["_line_"] = d[line_col].astype(str).fillna("")

    fig, ax = plt.subplots(figsize=(7, 6), dpi=110)
    for ln, dsub in d.groupby("_line_"):
        ax.scatter(dsub[lon_col].to_numpy(), dsub[lat_col].to_numpy(), s=s, alpha=0.7, label=str(ln))
    ax.set_xlabel(lon_col)
    ax.set_ylabel(lat_col)
    ax.set_title("Survey lines — overview")
    ax.grid(True, ls=":")
    if d["_line_"].nunique() <= 12:
        ax.legend(title="Line", fontsize=8)
    plt.show()


# ---------- interactive browser (map + time/distance series viewer) ----------
def make_line_browser(
    df,
    line_col="line",
    date_col="Reading_Date",
    time_col="Reading_Time",
    lat_col="Mag_Latitude",
    lon_col="Mag_Longitude",
    survey_col="Magnetic_Field",
    geomag_col=None,
    value_cols=None,
    # NEW: use your precomputed distance column
    distance_col="Profile_Dist_m",
    sort_distance=True,   # sort by distance when plotting distance profiles
):
    """
    Interactive browser to explore survey lines and plot vs Time (UTC) or precomputed Distance (m).

    Parameters
    ----------
    df : DataFrame
        Your survey data (already includes the precomputed distance column).
    line_col : str
        Column with line identifiers.
    date_col, time_col : str
        Columns for date and time; combined to a UTC-aware datetime if '__survey_datetime_utc' not present.
    lat_col, lon_col : str
        Latitude/Longitude columns for map view.
    survey_col : str
        Default primary channel to plot on the series panel.
    geomag_col : str or None
        Optional geomagnetic observatory total-field column; auto-guessed if None.
    value_cols : list[str] or None
        If None, inferred from numeric-like columns with sufficient valid values.
    distance_col : str, default "Profile_Dist_m"
        Column with along-track distance in meters (precomputed by you).
    sort_distance : bool, default True
        If True, sort the selected line by distance before plotting in Distance mode.
    """
    d = df.copy()
    d["__dt"] = _ensure_datetime(d, date_col=date_col, time_col=time_col)
    d[lat_col] = _to_num(d[lat_col]); d[lon_col] = _to_num(d[lon_col])
    d[survey_col] = _to_num(d[survey_col])

    # Default/guess geomag col
    if geomag_col is None:
        geomag_col = _guess_geomag_F_col(d)
    if geomag_col in d.columns:
        d[geomag_col] = _to_num(d[geomag_col])
    else:
        geomag_col = None

    # Clean for map & labels
    d_map = d.dropna(subset=[lat_col, lon_col]).copy()
    d["_line_"] = d[line_col].astype(str).fillna("")
    d_map["_line_"] = d_map[line_col].astype(str).fillna("")

    # Stable list of lines
    line_list = _ordered_unique(d["_line_"].tolist())
    if not line_list:
        raise ValueError(f"No lines found in '{line_col}'")

    # Build list of numeric-like columns for plotting choices
    if value_cols is None:
        drop_like = {
            line_col, date_col, time_col, "__dt", "__survey_datetime_utc",
            lat_col, lon_col, "Marker", "Marker_Time", "System_Time"
        }
        candidates = []
        for c in d.columns:
            if c in drop_like:
                continue
            s = pd.to_numeric(d[c].head(300), errors="coerce")
            if s.notna().mean() > 0.6:
                candidates.append(c)
        for defc in (survey_col, geomag_col):
            if defc and defc in d.columns and defc not in candidates:
                candidates.append(defc)
        value_cols = sorted(set(candidates))
    value_cols_with_none = ["(none)"] + value_cols

    # --- widgets ---
    # DROPDOWN (replaces SelectionSlider to avoid float index bug in Colab)
    slider = widgets.Dropdown(
        options=line_list,
        value=line_list[0],
        description="Line:",
        layout=widgets.Layout(width="40%")
    )

    axis_mode = widgets.ToggleButtons(
        options=[("Time (UTC)", "time"), ("Distance (m)", "dist")],
        value="time",
        description="X-axis:",
        layout=widgets.Layout(width="40%")
    )

    chA_dd = widgets.Dropdown(
        options=value_cols,
        value=survey_col if survey_col in value_cols else (value_cols[0] if value_cols else None),
        description="Channel A:", layout=widgets.Layout(width="45%")
    )
    chB_initial = geomag_col if (geomag_col and geomag_col in value_cols) else "(none)"
    chB_dd = widgets.Dropdown(
        options=value_cols_with_none, value=chB_initial,
        description="Channel B:", layout=widgets.Layout(width="45%")
    )

    first_ts = d[d["_line_"] == line_list[0]].dropna(subset=["__dt"])
    info = widgets.HTML(f"<b>Selected:</b> {line_list[0]} &nbsp; | &nbsp; <b>Date:</b> {_line_dates_label(first_ts)}")
    out = widgets.Output()

    def _prep_series(d_ts, mode):
        """
        Return (x, xlabel, d_plot) for the chosen x-axis mode.
        - 'time': x = '__dt'
        - 'dist': x = numeric 'distance_col' (precomputed), optionally sorted.
        """
        if mode == "time":
            d_plot = d_ts.copy()
            x = d_plot["__dt"]
            xlabel = "Time (UTC)"
            return x, xlabel, d_plot

        # distance mode (use your column)
        if distance_col not in d_ts.columns:
            # fallback to time if missing
            d_plot = d_ts.copy()
            x = d_plot["__dt"]
            return x, "Time (UTC)", d_plot

        d_plot = d_ts.copy()
        d_plot["__dist__"] = _to_num(d_plot[distance_col])
        # keep finite distances only for plotting
        d_plot = d_plot[np.isfinite(d_plot["__dist__"])]
        if sort_distance:
            d_plot = d_plot.sort_values("__dist__")
        x = d_plot["__dist__"].to_numpy()
        xlabel = f"Distance along line (m) [{distance_col}]"
        return x, xlabel, d_plot

    def redraw(selected_line, colA, colB, mode):
        with out:
            out.clear_output(wait=True)
            fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(11, 6.7), dpi=130, constrained_layout=True)

            # Data for this line (time-sorted for stable behavior)
            d_ts = d[d["_line_"] == selected_line].dropna(subset=["__dt"]).sort_values("__dt")
            date_label = _line_dates_label(d_ts)

            # --- Map ---
            ax0 = axes[0]
            ax0.scatter(d_map[lon_col], d_map[lat_col], s=2, label="All")
            d_line_map = d_map[d_map["_line_"] == selected_line]
            if not d_line_map.empty:
                ax0.scatter(d_line_map[lon_col], d_line_map[lat_col], s=6, label=selected_line)
            ax0.set_xlabel(lon_col); ax0.set_ylabel(lat_col)
            ax0.set_title(f"Track — Line: {selected_line}  |  Date: {date_label}")
            ax0.grid(True, ls=":"); ax0.legend(loc="best", fontsize=8)

            # --- Series (Time or Distance) ---
            ax1 = axes[1]
            if not d_ts.empty:
                xvals, xlabel, d_plot = _prep_series(d_ts, mode)
                if colA and colA in d_plot.columns:
                    ax1.plot(xvals, _to_num(d_plot[colA]), lw=1.2, label=colA)
                if colB and (colB != "(none)") and (colB in d_plot.columns):
                    ax1.plot(xvals, _to_num(d_plot[colB]), lw=1.0, alpha=0.9, label=colB)
                ax1.set_xlabel(xlabel)
            else:
                ax1.set_xlabel("Time (UTC)")

            ax1.set_ylabel("Value")
            ax1.set_title(f"{'Distance' if mode=='dist' else 'Time'} series — Line: {selected_line}  |  Date: {date_label}")
            ax1.grid(True, ls=":")
            ax1.legend(loc="best", fontsize=8)

            # avoid scientific notation on Y; keep integer ticks if suitable
            ax1.yaxis.set_major_formatter(mticker.ScalarFormatter(useMathText=False))
            ax1.ticklabel_format(style='plain', axis='y')
            ax1.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))

            plt.show()

    # Initial draw
    redraw(slider.value, chA_dd.value, chB_dd.value, axis_mode.value)

    # Callbacks
    def on_line_change(change):
        if change["name"] == "value" and change["type"] == "change":
            sel = change["new"]
            d_ts_sel = d[d["_line_"] == sel].dropna(subset=["__dt"])
            info.value = f"<b>Selected:</b> {sel} &nbsp; | &nbsp; <b>Date:</b> {_line_dates_label(d_ts_sel)}"
            redraw(sel, chA_dd.value, chB_dd.value, axis_mode.value)

    def on_chA_change(change):
        if change["name"] == "value":
            redraw(slider.value, change["new"], chB_dd.value, axis_mode.value)

    def on_chB_change(change):
        if change["name"] == "value":
            redraw(slider.value, chA_dd.value, change["new"], axis_mode.value)

    def on_axis_mode_change(change):
        if change["name"] == "value":
            redraw(slider.value, chA_dd.value, chB_dd.value, change["new"])

    slider.observe(on_line_change, names="value")
    chA_dd.observe(on_chA_change, names="value")
    chB_dd.observe(on_chB_change, names="value")
    axis_mode.observe(on_axis_mode_change, names="value")

    # Layout
    controls_top = HBox([slider, axis_mode, info])
    controls_bottom = HBox([chA_dd, chB_dd])
    display(VBox([controls_top, controls_bottom, out]))

In [5]:
df_filtered.columns

Index(['Reading_Date', 'Reading_Time', 'Magnetic_Field', 'Signal_Quality',
       'Depth', 'Leak', 'Measurement_Time', 'Signal_Strength',
       'Gradient_Condition', 'Weak_Signal', 'Poor_Reading',
       'Instrument_Mistuned', 'Reading_ID', 'System_Time', 'Line_Name',
       'Mag_Latitude', 'Mag_Longitude', 'Mag_Easting', 'Mag_Northing',
       'Mag_Zone', 'GPS_Latitude', 'GPS_Longitude', 'GPS_Easting',
       'GPS_Northing', 'GPS_Zone', 'Marker', 'Marker_Time', 'timestamp_s',
       'source_file', '__survey_datetime_utc', 'DATE_vss', 'TIME_vss',
       'DOY_vss', 'VSSX_vss', 'VSSY_vss', 'VSSZ_vss', 'VSSF_vss',
       'Profile_Dist_m'],
      dtype='object')

In [6]:
make_line_browser(df_raw,
                  line_col="Line_Name",
                  date_col="Reading_Date", time_col="Reading_Time",
                  lat_col="Mag_Northing", lon_col="Mag_Easting",
                  survey_col="Magnetic_Field",
                  geomag_col="VSSF_vss",
                  distance_col="Profile_Dist_m"
                  )

In [7]:
make_line_browser(df_filtered,
                  line_col="Line_Name",
                  date_col="Reading_Date", time_col="Reading_Time",
                  lat_col="Mag_Northing", lon_col="Mag_Easting",
                  survey_col="Magnetic_Field",
                  geomag_col="VSSF_vss",
                  distance_col="Profile_Dist_m"
                  )

# 🧹 De-spiking Magnetic Survey Time Series with a Rolling Hampel Filter

## 🎯 Objective
Magnetic survey data often contains short, high-amplitude **spikes** (navigation glitches, telemetry dropouts, EM interference).
These spikes distort the true magnetic field and can bias later steps (e.g., diurnal correction, leveling).
We apply a **Hampel filter** (rolling **median** + **MAD**) to detect spikes, then **replace** them with one of several interpolation modes.

---

## ⚙️ Method Overview
The Hampel detector is robust for geophysical series: it adapts locally to noise and ignores isolated spikes.
After flagging, we replace spike samples **in time** using:

- **Time-linear interpolation** → `replace="interp"` or `"linear"`
- **Rolling median + interpolation** → `replace="median"`
- **Local polynomial interpolation in time** → `replace="poly"` (new)

**Polynomial mode (new):** fits a **local least-squares polynomial** of degree `poly_order`
using `poly_neighbors` nearby valid samples (≈ half on each side), then evaluates it at spike times.
This gives smoother fills across short gaps while staying local and numerically stable.

---

## 🔢 Algorithmic Concept (with formulas for each implemented method)

### 1) Spike detection via rolling Hampel filter
Given a series $\{x_i\}$ and a rolling window $\mathcal{W}_i$ centered at $i$:

1. **Rolling median**
   $
   m_i \;=\; \mathrm{median}\{\,x_j : j \in \mathcal{W}_i\,\}
   $

2. **Median Absolute Deviation (MAD)**
   $
   \mathrm{MAD}_i \;=\; \mathrm{median}\{\,|x_j - m_i| : j \in \mathcal{W}_i\,\}
   $

3. **Robust sigma estimate**
   $
   \sigma_i \;=\; 1.4826 \times \mathrm{MAD}_i
   $

4. **Spike rule**
   $
   \text{flag}_i \;=\; \bigl(|x_i - m_i| > T\,\sigma_i \bigr)
   $
   with $T \in [3,4]$ (typical: $T=3.5$).

Let $\mathcal{S}=\{\,i : \text{flag}_i=\text{True}\,\}$ be the set of indices to replace.

---

### 2) Replacement algorithms

#### A) Time-linear interpolation (`replace="interp"` or `"linear"`)
Let $t_i$ denote the timestamp (in seconds) at index $i$.
For a flagged point $i \in \mathcal{S}$, find the nearest **valid** neighbors
$L < i < R$ with values $x_L, x_R$ at times $t_L, t_R$.

**Linear in time**
$
\displaystyle
\hat{x}_i \;=\; x_L \;+\; (x_R - x_L)\,\frac{t_i - t_L}{\,t_R - t_L\,}
$

- Works on slightly irregular sampling because it uses $t$ explicitly.
- If only one neighbor exists (edge of series), the method does **not** extrapolate;
  remaining NaNs can be handled by optional fallbacks (see poly mode).

---

#### B) Rolling median + interpolation (`replace="median"`)
1. Prefill spikes with the local median:
   $
   x_i^\star \;=\;
   \begin{cases}
     m_i, & i \in \mathcal{S}\\[2pt]
     x_i, & \text{otherwise}
   \end{cases}
   $
2. Smooth any residual transitions via time-linear interpolation (as in A).

- More robust on longer gaps or dense bursts; may smooth sharp geological features slightly more than A.

---

#### C) Local polynomial interpolation (`replace="poly"`)
For each $i \in \mathcal{S}$:

1. Choose a **local neighborhood** of valid indices $\mathcal{N}_i$ of size
   $\text{neighbors} = N \ge p+1$ (where $p=\text{polynomial order}$), split roughly half to the left and right of $i$.

2. Use **centered time** for numerical stability.
   With $t_i$ the spike time and $\{t_k\}_{k\in\mathcal{N}_i}$ the neighbor times, define
   $
   \tilde{t}_k \;=\; t_k - \bar{t}\,, \quad
   \bar{t} \;=\; \frac{1}{|\mathcal{N}_i|}\sum_{k\in\mathcal{N}_i} t_k\,, \quad
   \tilde{t}_i \;=\; t_i - \bar{t}.
   $

3. **Least-squares polynomial fit** of degree $p$ to neighbor pairs $(\tilde{t}_k, x_k)$:
   find coefficients $\mathbf{a} = (a_0,\dots,a_p)$ minimizing
   $
   \sum_{k\in\mathcal{N}_i}\Bigl[x_k - \sum_{\ell=0}^p a_\ell\, \tilde{t}_k^\ell\Bigr]^2.
   $

4. Evaluate at $\tilde{t}_i$:
   $
   \hat{x}_i \;=\; \sum_{\ell=0}^p a_\ell\, \tilde{t}_i^\ell.
   $

5. **Fallbacks**
   - If $|\mathcal{N}_i| < p+1$ or the fit is ill-conditioned, revert to **time-linear** (method A) using the nearest left/right valid neighbors.
   - After processing all spikes, if any edges remain NaN and `poly_edge_fallback=True`, apply a final time-linear interpolation across the series.

- Recommended settings at $2\,\text{Hz}$: $p=2$–$3$, $N=8$–$12$.  
- Delivers smoother fills across short gaps (1–3 s) than pure linear, while staying local.

---

## 🧭 Practical Parameters

| Parameter | Description | Typical / Recommended |
|---|---|---|
| `window` | Rolling window (seconds like `"5s"` or integer samples) | 2–7 s (e.g., `"5s"` for 2 Hz) |
| `threshold` | Spike sensitivity (robust $\sigma$) | $3.0$–$4.0$ (start **$3.5$**) |
| `replace` | Replacement mode | `"interp"`/`"linear"`, `"median"`, **`"poly"` |
| `poly_order` | Polynomial degree (poly mode) | $2$–$3$ (start **$3$**) |
| `poly_neighbors` | Total neighbors per gap (≈ half per side) | $8$–$12$ at $2\,\text{Hz}$ (start **$8$**) |
| `poly_edge_fallback` | After poly, linearly fill remaining edge NaNs | `True` |
| `add_mask_cols` | Add `{col}__spike` mask for QA | `True` |
| `output_suffix` | Suffix for cleaned series | `"_despiked"` (or `"_poly"`) |
| `tz` | Parse timezone (converted to UTC internally) | `"UTC"` |

**Notes**
- Ensure $\text{poly\_neighbors} \ge \text{poly\_order} + 1$ and keep $\text{poly\_order}$ modest to avoid overfit.
- All interpolation is **time-aware** via timestamps $t$, so slightly irregular sampling is handled correctly.

---

## 🧪 Suggested Presets (2 Hz sampling, $\Delta t = 0.5\,\text{s}$)

- **Balanced (first try)**  
  `window="5s"`, `threshold=3.5`, `replace="interp"`

- **Smoother fills across short gaps**  
  `window="5s"`, `threshold=3.5`, `replace="poly"`, `poly_order=3`, `poly_neighbors=8`

- **Conservative (clean lines)**  
  `window="7.5s"`, `threshold=4.2`, `replace="interp"`

- **Aggressive (noisy/bursty)**  
  `window="3s"`, `threshold=3.0`, `replace="poly"`, `poly_order=2`, `poly_neighbors=10`

- **Two-stage (bursts then residuals)**  
  1) `window="2s"`, `threshold=3.2`, `replace="interp"`  
  2) `window="6s"`, `threshold=3.8`, `replace="poly"`, `poly_order=3`, `poly_neighbors=10`


In [8]:
import numpy as np
import pandas as pd

def despike_survey(
    df,
    value_cols=("Magnetic_Field",),
    date_col="Reading_Date",
    time_col="Reading_Time",
    date_fmt="%d-%b-%Y %H:%M:%S.%f",
    tz="UTC",
    window="5s",              # time-based ("2s", "10s") or integer samples
    threshold=3.5,            # Hampel threshold (robust sigma)
    replace="interp",         # "interp"/"linear" (time), "median", or "poly"
    add_mask_cols=True,
    output_suffix="_despiked",
    # --- polynomial interpolation controls ---
    poly_order=3,             # polynomial degree (2–5 typical)
    poly_neighbors=8,         # total neighbors (~half per side); must be >= poly_order+1
    poly_edge_fallback=True   # fill remaining edge NaNs with time-linear
):
    """
    Despike one or more survey signals using a Hampel-style filter
    (rolling median + MAD), preserving original row order and handling
    NaT timestamps safely.

    The function builds UTC timestamps per row (using an existing
    ``__survey_datetime_utc`` column if present; otherwise parsing
    ``date_col`` + ``time_col``), sorts valid-timestamp rows by time,
    detects spikes via a rolling median + MAD (Hampel filter), and
    replaces flagged samples using one of:

    - ``"interp"`` / ``"linear"``: time-based linear interpolation
      (uses the DatetimeIndex; slight irregular timing OK).
    - ``"median"``: fill spikes with the rolling median, then apply
      time-linear interpolation to smooth transitions.
    - ``"poly"``: **local polynomial interpolation in time** with degree
      ``poly_order`` using ``poly_neighbors`` nearby valid samples; any
      remaining edge NaNs can be linearly filled if
      ``poly_edge_fallback=True``.

    Parameters
    ----------
    df : pandas.DataFrame
        Input survey table. Must contain either ``__survey_datetime_utc``
        (timezone-aware) or both ``date_col`` and ``time_col`` to build
        timestamps. Must also contain all columns listed in ``value_cols``.
    value_cols : sequence of str, default ("Magnetic_Field",)
        Names of numeric columns to despike.
    date_col : str, default "Reading_Date"
        Date column used when ``__survey_datetime_utc`` is absent.
    time_col : str, default "Reading_Time"
        Time column used when ``__survey_datetime_utc`` is absent.
    date_fmt : str, default "%d-%b-%Y %H:%M:%S.%f"
        Format string for parsing ``date_col`` + ``time_col`` if needed.
    tz : str, default "UTC"
        If not "UTC", parsed times are converted *from* this timezone to UTC.
        Ignored when ``__survey_datetime_utc`` already exists.
    window : str or int, default "5s"
        Rolling window for Hampel stats:
        - If str (e.g., "2s", "10s"), uses a time-based window on the
          DatetimeIndex.
        - If int, uses a centered, sample-count window (min 3 samples).
    threshold : float, default 3.5
        Spike threshold in robust-sigma units where
        ``sigma = 1.4826 * MAD`` computed in the rolling window.
    replace : {"interp","linear","median","poly"}, default "interp"
        Replacement strategy for flagged spikes.
    add_mask_cols : bool, default True
        If True, add a boolean mask column per signal: ``f"{col}__spike"``.
    output_suffix : str, default "_despiked"
        Suffix for cleaned output columns, e.g., ``"Magnetic_Field_despiked"``.
    poly_order : int, default 3
        Polynomial degree for local time interpolation when ``replace="poly"``.
        Keep modest (2–3) unless gaps are tiny and signal very smooth.
    poly_neighbors : int, default 8
        Total nearby valid samples used per NaN (≈ half per side). Must be
        ``>= poly_order + 1``. For 2 Hz data, 8–12 is a good range.
    poly_edge_fallback : bool, default True
        After polynomial filling, apply a final
        ``interpolate(method="time", limit_direction="both")`` to fill any
        remaining edge NaNs.

    Returns
    -------
    pandas.DataFrame
        Copy of ``df`` with added columns:
        - For each ``col`` in ``value_cols``:
          ``f"{col}{output_suffix}"`` with the despiked signal.
        - If ``add_mask_cols``: ``f"{col}__spike"`` (bool) marking spikes.

    Notes
    -----
    - Rows with NaT timestamps are skipped and yield NaNs in outputs.
    - For time-based windows, the function temporarily reindexes valid rows
      on a UTC DatetimeIndex to enable ``.rolling("Xs")``.
    - Normalize instrument sentinels (e.g., 88888/99999) to NaN *before*
      calling if present in your data columns.
    - Two-pass approach can help with bursty noise: first a short window
      (e.g., "2s", threshold ~3.2), then a slightly longer window
      (e.g., "6s", threshold ~3.8).

    Examples
    --------
    >>> # Balanced settings for 2 Hz data:
    >>> df_clean = despike_survey(
    ...     df, value_cols=("Magnetic_Field",),
    ...     window="5s", threshold=3.5, replace="interp"
    ... )
    >>> # Polynomial interpolation (smoother across short gaps):
    >>> df_poly = despike_survey(
    ...     df, value_cols=("Magnetic_Field",),
    ...     window="5s", threshold=3.5,
    ...     replace="poly", poly_order=3, poly_neighbors=8
    ... )
    """
    d = df.copy()

    # ---- Build UTC datetime (NaT-safe) ----
    if "__survey_datetime_utc" in d.columns:
        dt_all = pd.to_datetime(d["__survey_datetime_utc"], errors="coerce", utc=True)
    else:
        dt_all = pd.to_datetime(
            d[date_col].astype(str) + " " + d[time_col].astype(str),
            format=date_fmt, errors="coerce", utc=True
        )
    if tz and tz.upper() != "UTC":
        dt_all = dt_all.dt.tz_convert("UTC")

    # Work only on rows with valid time
    valid = dt_all.notna()
    if not valid.any():
        raise ValueError("All survey timestamps are NaT; check date/time parsing.")

    # Time-sorted working frame with DatetimeIndex
    w = d.loc[valid].copy()
    w.index = pd.DatetimeIndex(dt_all[valid])
    w = w.sort_index()

    # Rolling accessor (time-based or sample-based)
    def _rolling(s: pd.Series):
        if isinstance(window, str):
            return s.rolling(window, min_periods=1)
        else:
            wlen = max(int(window or 3), 3)
            return s.rolling(wlen, center=True, min_periods=1)

    # Local polynomial filler (DatetimeIndex-aware)
    def _poly_interpolate_time(series: pd.Series, order=3, neighbors=8, edge_fallback=True) -> pd.Series:
        s = pd.to_numeric(series, errors="coerce")
        idx = s.index  # DatetimeIndex
        t = idx.asi8.astype("float64") / 1e9  # seconds since epoch (float)

        y = s.to_numpy(dtype="float64", copy=True)
        good = np.isfinite(y)
        if good.sum() < max(order + 1, 2):
            # Not enough valid points overall: just do time-linear
            return s.interpolate(method="time", limit_direction="both")

        vi = np.flatnonzero(good)
        neighbors = max(int(neighbors), order + 1)
        left_n = neighbors // 2
        right_n = neighbors - left_n

        nan_pos = np.flatnonzero(~good)
        for i in nan_pos:
            j = np.searchsorted(vi, i)
            left_idx = vi[max(0, j - left_n): j]
            right_idx = vi[j: j + right_n]
            sel = np.unique(np.concatenate([left_idx, right_idx]))

            if sel.size < order + 1:
                # fallback: simple 2-point linear if possible
                l = vi[j - 1] if j - 1 >= 0 else None
                r = vi[j] if j < vi.size else None
                if l is not None and r is not None and t[r] != t[l]:
                    y[i] = y[l] + (y[r] - y[l]) * (t[i] - t[l]) / (t[r] - t[l])
                continue

            tfit = t[sel]; yfit = y[sel]
            t0 = tfit.mean()
            xfit = tfit - t0
            xn = t[i] - t0
            try:
                coef = np.polyfit(xfit, yfit, order)
                y[i] = np.polyval(coef, xn)
            except np.linalg.LinAlgError:
                # ill-conditioned: fallback to local linear if possible
                l = left_idx[-1] if left_idx.size else None
                r = right_idx[0] if right_idx.size else None
                if l is not None and r is not None and t[r] != t[l]:
                    y[i] = y[l] + (y[r] - y[l]) * (t[i] - t[l]) / (t[r] - t[l])

        s_out = pd.Series(y, index=idx)
        if edge_fallback and s_out.isna().any():
            s_out = s_out.interpolate(method="time", limit_direction="both")
        return s_out

    # ---------------- Process each signal ----------------
    for col in value_cols:
        if col not in w.columns:
            continue
        x = pd.to_numeric(w[col], errors="coerce")

        med  = _rolling(x).median()
        absd = (x - med).abs()
        rmad = _rolling(absd).median()

        sigma = 1.4826 * rmad.replace(0, np.nan)
        z = absd / sigma
        spike = (z > threshold).fillna(False)

        y = x.copy()
        y[spike] = np.nan

        rep = str(replace).lower()
        if rep in ("interp", "linear", "time"):
            y = y.interpolate(method="time", limit_direction="both")
        elif rep == "median":
            y = y.fillna(med).interpolate(method="time", limit_direction="both")
        elif rep == "poly":
            y = _poly_interpolate_time(
                y, order=poly_order, neighbors=poly_neighbors, edge_fallback=poly_edge_fallback
            )
        else:
            raise ValueError("replace must be one of: 'interp'/'linear', 'median', 'poly'")

        # Put results back in original row order
        new_y = pd.Series(index=w.index, data=y)
        new_spike = pd.Series(index=w.index, data=spike)

        # Reindex to the original (valid) rows’ order
        orig_order_idx = pd.DatetimeIndex(dt_all[valid])
        d.loc[valid, f"{col}{output_suffix}"] = new_y.reindex(orig_order_idx).values
        if add_mask_cols:
            d.loc[valid, f"{col}__spike"] = new_spike.reindex(orig_order_idx).values

    return d

In [9]:
df_poly = despike_survey(
    df_filtered,
    value_cols=("Magnetic_Field",),
    window="5s",
    threshold=3.5,
    replace="poly",     # local polynomial in time
    poly_order=3,
    poly_neighbors=8,
    add_mask_cols=True,
    output_suffix="_poly"
)

In [10]:
df_interp = despike_survey(
    df_filtered,
    value_cols=("Magnetic_Field",),
    window="3s",
    threshold=3.1,
    replace="interp",     # local polynomial in time
    # poly_order=3,
    # poly_neighbors=8,
    add_mask_cols=True,
    output_suffix="_interp"
)

In [11]:
df_poly.columns

Index(['Reading_Date', 'Reading_Time', 'Magnetic_Field', 'Signal_Quality',
       'Depth', 'Leak', 'Measurement_Time', 'Signal_Strength',
       'Gradient_Condition', 'Weak_Signal', 'Poor_Reading',
       'Instrument_Mistuned', 'Reading_ID', 'System_Time', 'Line_Name',
       'Mag_Latitude', 'Mag_Longitude', 'Mag_Easting', 'Mag_Northing',
       'Mag_Zone', 'GPS_Latitude', 'GPS_Longitude', 'GPS_Easting',
       'GPS_Northing', 'GPS_Zone', 'Marker', 'Marker_Time', 'timestamp_s',
       'source_file', '__survey_datetime_utc', 'DATE_vss', 'TIME_vss',
       'DOY_vss', 'VSSX_vss', 'VSSY_vss', 'VSSZ_vss', 'VSSF_vss',
       'Profile_Dist_m', 'Magnetic_Field_poly', 'Magnetic_Field__spike'],
      dtype='object')

In [12]:
df_interp.columns

Index(['Reading_Date', 'Reading_Time', 'Magnetic_Field', 'Signal_Quality',
       'Depth', 'Leak', 'Measurement_Time', 'Signal_Strength',
       'Gradient_Condition', 'Weak_Signal', 'Poor_Reading',
       'Instrument_Mistuned', 'Reading_ID', 'System_Time', 'Line_Name',
       'Mag_Latitude', 'Mag_Longitude', 'Mag_Easting', 'Mag_Northing',
       'Mag_Zone', 'GPS_Latitude', 'GPS_Longitude', 'GPS_Easting',
       'GPS_Northing', 'GPS_Zone', 'Marker', 'Marker_Time', 'timestamp_s',
       'source_file', '__survey_datetime_utc', 'DATE_vss', 'TIME_vss',
       'DOY_vss', 'VSSX_vss', 'VSSY_vss', 'VSSZ_vss', 'VSSF_vss',
       'Profile_Dist_m', 'Magnetic_Field_interp', 'Magnetic_Field__spike'],
      dtype='object')

In [13]:
make_line_browser(df_poly,
                  line_col="Line_Name",
                  date_col="Reading_Date", time_col="Reading_Time",
                  lat_col="Mag_Northing", lon_col="Mag_Easting",
                  survey_col="Magnetic_Field_poly",
                  geomag_col="Magnetic_Field",
                  distance_col="Profile_Dist_m"
                  )

In [14]:
make_line_browser(df_interp,
                  line_col="Line_Name",
                  date_col="Reading_Date", time_col="Reading_Time",
                  lat_col="Mag_Northing", lon_col="Mag_Easting",
                  survey_col="Magnetic_Field_interp",
                  geomag_col="Magnetic_Field",
                  distance_col="Profile_Dist_m"
                  )

## Diurnal Variation Correction

## 🌍 Diurnal Variation Correction — Projection Toward the IGRF

## 🎯 Objective
The goal of the **diurnal correction** is to remove temporal variations in the measured magnetic field that are caused by fluctuations in the Earth's main field and ionospheric–magnetospheric currents rather than by geological sources.

These temporal changes (typically up to tens of nT per hour) are monitored continuously by **geomagnetic observatories**, which record the field components **X (north)**, **Y (east)**, and **Z (down)**.  
By combining these records with the **International Geomagnetic Reference Field (IGRF)** model, we can project the observed vector variations along the direction of the main geomagnetic field at each survey location, producing a correction that matches the orientation of the local total field measurement.

---

## ⚙️ Method Overview

1. **Observatory data** provide the time-varying vector components:  
   $X_\text{obs}(t),\; Y_\text{obs}(t),\; Z_\text{obs}(t)$

2. For each UTC day, compute the **daily baseline** (usually the median):  
   $$
   X_b = \text{median}[X_\text{obs}(t)], \quad
   Y_b = \text{median}[Y_\text{obs}(t)], \quad
   Z_b = \text{median}[Z_\text{obs}(t)]
   $$

3. The **diurnal disturbance** is the deviation from this baseline:  
   $$
   \Delta X = X_\text{obs} - X_b, \quad
   \Delta Y = Y_\text{obs} - Y_b, \quad
   \Delta Z = Z_\text{obs} - Z_b
   $$

4. For each survey point, compute the **unit vector** of the IGRF main field  
   at its position $(\phi, \lambda, h)$ and time $t$:  
   $$
   \hat{\mathbf{B}}_\text{IGRF}(t) =
   \frac{(X_\text{IGRF}, Y_\text{IGRF}, Z_\text{IGRF})}
        {\sqrt{X_\text{IGRF}^2 + Y_\text{IGRF}^2 + Z_\text{IGRF}^2}}
   $$

5. The **projected diurnal variation** at the survey site is:  
   $$
   \Delta F_\text{proj}(t) =
   \Delta X(t)\,\hat{B}_x +
   \Delta Y(t)\,\hat{B}_y +
   \Delta Z(t)\,\hat{B}_z
   $$

6. The **corrected total field** becomes:  
   $$
   F_\text{corr}(t) = F_\text{measured}(t) - \Delta F_\text{proj}(t)
   $$

---

## 🧭 Why Projection Matters
Magnetometers measure the **total magnetic field intensity** along the local direction of the main field, not the individual X, Y, or Z components.  
Directly subtracting scalar observatory data can misrepresent the correction if the observatory is at a different magnetic latitude.  
The **projection method** ensures the diurnal variation is applied along the correct direction of the field at the survey site, improving accuracy in low-latitude or inclined-field regions.

---

## ⚡ Implementation Notes
- **IGRF model** computed via `ppigrf` (or alternatives like `pyIGRF`, `igrf`, or `geomagpy`), returning main field components **(X, Y, Z)** in nT.  
- **Altitude** is expressed in meters (positive upward).  
- **Baselines** are calculated per UTC day using either *median* (robust) or *mean* (smoother).  
- The projection is applied sample-by-sample using the IGRF direction for each location and time.

---

## 📚 Key References

- **Clark, D. A. (2014)**. *Methods for the reduction, processing and interpretation of aeromagnetic data.*  
  *Geophysical Prospecting*, **62(3)**, 505–533.  
  DOI: [10.1111/1365-2478.12100](https://doi.org/10.1111/1365-2478.12100)

- **Reeves, C. V. (2005)**. *Aeromagnetic Surveys: Principles, Practice and Interpretation.*  
  *Geosoft Technical Paper Series.*

- **Moses, D., & Oldenburg, D. W. (2018)**. *Understanding diurnal corrections in magnetic data.*  
  *UBC-GIF Technical Note.*

- **Merrill, R. T., McElhinny, M. W., & McFadden, P. L. (1996)**. *The Magnetic Field of the Earth: Paleomagnetism, the Core, and the Deep Mantle.*  
  Academic Press.

- **Finlay, C. C. et al. (2020)**. *International Geomagnetic Reference Field: the thirteenth generation (IGRF-13).*  
  *Earth, Planets and Space*, **72**, 49.  
  DOI: [10.1186/s40623-020-01288-x](https://doi.org/10.1186/s40623-020-01288-x)

---

## 🧠 Summary
By projecting the vector diurnal variation from observatory records onto the IGRF direction at each survey point, we obtain a correction that is both **physically consistent** and **spatially transferable**.  
This approach minimizes bias in **marine and airborne magnetic surveys**, particularly where field inclination and declination differ between the observatory and the survey area.

In [15]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

# ---------- helpers ----------
def _decimal_year_to_timestamp(dy):
    dy = np.asarray(dy, dtype=float)
    year = np.floor(dy).astype(int)
    frac = dy - year
    starts = pd.to_datetime([f"{y}-01-01T00:00:00Z" for y in year], utc=True)
    ends   = pd.to_datetime([f"{y+1}-01-01T00:00:00Z" for y in year], utc=True)
    dt = starts + (ends - starts) * frac
    return dt if dt.size > 1 else dt[0]

def _ensure_survey_time(df, date_col="Reading_Date", time_col="Reading_Time"):
    if "__survey_datetime_utc" in df.columns:
        return pd.to_datetime(df["__survey_datetime_utc"], errors="coerce", utc=True)
    return pd.to_datetime(
        df[date_col].astype(str) + " " + df[time_col].astype(str),
        format="%d-%b-%Y %H:%M:%S.%f", errors="coerce", utc=True
    )

def _nearest_time_index(survey_time_utc, grid_times_utc, tolerance="365D"):
    """Return integer positions into `grid_times_utc` for each survey time, or -1 if NaT/out of tol."""
    s = pd.DataFrame({"i": np.arange(len(survey_time_utc)),
                      "time": pd.to_datetime(survey_time_utc, utc=True, errors="coerce")})
    g = pd.DataFrame({"time": pd.Index(pd.to_datetime(grid_times_utc, utc=True)).sort_values().unique()})\
            .reset_index().rename(columns={"index":"g_idx"})
    out = np.full(len(s), -1, dtype=int)
    s_valid = s[s["time"].notna()].sort_values("time")
    if s_valid.empty or g.empty:
        return out
    joined = pd.merge_asof(s_valid, g, on="time", direction="nearest",
                           tolerance=pd.Timedelta(tolerance))
    gid = joined["g_idx"].to_numpy()
    ok = np.isfinite(gid)
    out[s_valid.loc[ok, "i"].to_numpy()] = gid[ok].astype(int)
    return out

# ---------- 1) ATTACH IGRF TO SURVEY (nearest time + nearest spatial neighbor) ----------
def attach_igrf_to_survey(
    df_survey, df_igrf,
    lat_col="Mag_Latitude", lon_col="Mag_Longitude",
    date_col="Reading_Date", time_col="Reading_Time",
    time_tolerance="365D"
):
    out = df_survey.copy()
    # timestamps
    dt = _ensure_survey_time(out, date_col, time_col)
    out["__dt_utc__"] = dt

    # grid unique times
    grid_times = pd.Index(pd.to_datetime(df_igrf["time"].dropna().unique(), utc=True)).sort_values()
    if len(grid_times) == 0:
        raise RuntimeError("IGRF grid has no time values.")

    # map each survey row to a grid time position
    t_idx = _nearest_time_index(dt, grid_times, tolerance=time_tolerance)

    # columns to fill
    cols = ["IGRF_time","IGRF_X","IGRF_Y","IGRF_Z","IGRF_F","IGRF_Dec","IGRF_Inc","IGRF_bx","IGRF_by","IGRF_bz",
            "IGRF_nn_dist_deg"]
    for c in cols:
        out[c] = np.nan
    out["IGRF_time"] = pd.NaT

    # group survey positions by grid time
    groups = {}
    for pos, gi in enumerate(t_idx):
        if gi >= 0:
            groups.setdefault(int(gi), []).append(pos)

    # process each time slice
    for gi, rows in groups.items():
        t_sel = pd.to_datetime(grid_times[gi])

        gslice = df_igrf[df_igrf["time"] == t_sel]
        if gslice.empty:
            continue

        # finite grid coords
        glat = pd.to_numeric(gslice["lat"], errors="coerce").to_numpy()
        glon = pd.to_numeric(gslice["lon"], errors="coerce").to_numpy()
        gmask = np.isfinite(glat) & np.isfinite(glon)
        if not np.any(gmask):
            continue

        G = np.column_stack([glat[gmask], glon[gmask]])
        tree = cKDTree(G)

        # survey coords
        qlat_all = pd.to_numeric(out.iloc[rows][lat_col], errors="coerce").to_numpy()
        qlon_all = pd.to_numeric(out.iloc[rows][lon_col], errors="coerce").to_numpy()
        qmask = np.isfinite(qlat_all) & np.isfinite(qlon_all)
        if not np.any(qmask):
            continue

        Q = np.column_stack([qlat_all[qmask], qlon_all[qmask]])
        dist, j = tree.query(Q, k=1, workers=-1)

        # slice grid fields, apply same gmask
        def _get(name):
            return pd.to_numeric(gslice[name], errors="coerce").to_numpy()[gmask] if name in gslice.columns else None

        Xg, Yg, Zg = _get("X"), _get("Y"), _get("Z")
        Fg, Dg, Ig = _get("F"), _get("Dec"), _get("Inc")

        # if X/Y/Z missing, reconstruct from F/Dec/Inc
        if Xg is None or Yg is None or Zg is None:
            if Fg is None or Dg is None or Ig is None:
                # no usable field info
                continue
            Dd = np.deg2rad(Dg); Id = np.deg2rad(Ig)
            Xg = Fg*np.cos(Id)*np.cos(Dd)
            Yg = Fg*np.cos(Id)*np.sin(Dd)
            Zg = Fg*np.sin(Id)

        # nearest values for valid queries
        Xq, Yq, Zq = Xg[j], Yg[j], Zg[j]
        B = np.sqrt(Xq*Xq + Yq*Yq + Zq*Zq)
        ok = np.isfinite(B) & (B > 0)

        bx = np.full_like(B, np.nan, dtype=float)
        by = np.full_like(B, np.nan, dtype=float)
        bz = np.full_like(B, np.nan, dtype=float)
        bx[ok] = Xq[ok]/B[ok]; by[ok] = Yq[ok]/B[ok]; bz[ok] = Zq[ok]/B[ok]

        # write back only to rows with finite survey coords
        sel = np.asarray(rows, dtype=int)[qmask]
        col = out.columns.get_indexer
        out.iloc[sel, col(["IGRF_time"])] = pd.to_datetime([t_sel]*len(sel), utc=True)  # broadcast safely
        out.iloc[sel, col(["IGRF_X"])]    = Xq.reshape(-1,1)
        out.iloc[sel, col(["IGRF_Y"])]    = Yq.reshape(-1,1)
        out.iloc[sel, col(["IGRF_Z"])]    = Zq.reshape(-1,1)
        # conditionally set F/Dec/Inc if present in grid slice
        if "IGRF_F" in out.columns and "F" in df_igrf.columns:
            out.iloc[sel, col(["IGRF_F"])] = Fg[j].reshape(-1,1)
        if "IGRF_Dec" in out.columns and "Dec" in df_igrf.columns:
            out.iloc[sel, col(["IGRF_Dec"])] = Dg[j].reshape(-1,1)
        if "IGRF_Inc" in out.columns and "Inc" in df_igrf.columns:
            out.iloc[sel, col(["IGRF_Inc"])] = Ig[j].reshape(-1,1)
        out.iloc[sel, col(["IGRF_bx"])]  = bx.reshape(-1,1)
        out.iloc[sel, col(["IGRF_by"])]  = by.reshape(-1,1)
        out.iloc[sel, col(["IGRF_bz"])]  = bz.reshape(-1,1)
        out.iloc[sel, col(["IGRF_nn_dist_deg"])] = dist.reshape(-1,1)

    return out

# ---------- 2) PROJECTION + CORRECTION using attached IGRF ----------
def apply_projection_and_correction(
    df_with_igrf,
    x_col="VSSX_vss", y_col="VSSY_vss", z_col="VSSZ_vss",
    field_col="Magnetic_Field_despiked",
    baseline="median"
):
    out = df_with_igrf.copy()
    dt = pd.to_datetime(out["__dt_utc__"], errors="coerce", utc=True)

    # clean observatory XYZ (remove sentinel values)
    Xo = pd.to_numeric(out.get(x_col), errors="coerce").replace({88888.0: np.nan, 99999.0: np.nan})
    Yo = pd.to_numeric(out.get(y_col), errors="coerce").replace({88888.0: np.nan, 99999.0: np.nan})
    Zo = pd.to_numeric(out.get(z_col), errors="coerce").replace({88888.0: np.nan, 99999.0: np.nan})

    # compute daily baseline per component on the full series (aligns automatically)
    idx = pd.DatetimeIndex(dt).tz_convert("UTC")
    def _baseline(s):
        s = pd.Series(s.values, index=idx)
        day = s.index.normalize()
        if baseline == "median":
            return s - s.groupby(day).transform("median")
        else:
            return s - s.groupby(day).transform("mean")

    dX = _baseline(Xo)
    dY = _baseline(Yo)
    dZ = _baseline(Zo)

    # projection (needs finite dX/dY/dZ and finite bx/by/bz)
    bx = pd.to_numeric(out["IGRF_bx"], errors="coerce")
    by = pd.to_numeric(out["IGRF_by"], errors="coerce")
    bz = pd.to_numeric(out["IGRF_bz"], errors="coerce")

    ok = dt.notna() & dX.notna() & dY.notna() & dZ.notna() & bx.notna() & by.notna() & bz.notna()
    proj = np.full(len(out), np.nan, dtype=float)
    proj[ok] = dX[ok].to_numpy()*bx[ok].to_numpy() + dY[ok].to_numpy()*by[ok].to_numpy() + dZ[ok].to_numpy()*bz[ok].to_numpy()

    out["diurnal_proj_nT"] = proj
    out[f"{field_col}_corr"] = pd.to_numeric(out[field_col], errors="coerce") - out["diurnal_proj_nT"]

    # quick diagnostics
    print(f"Attached IGRF on {out['IGRF_time'].notna().sum()} / {len(out)} rows "
          f"(time tol matched).")
    print(f"Finite bx/by/bz on {np.isfinite(bx).sum()} / {len(out)} rows.")
    print(f"Projected rows: {np.isfinite(proj).sum()} / {len(out)}.")

    return out

In [16]:
# ============================================================
# IGRF grid ingest + attach to survey + projection correction
# ============================================================

import os
import re
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.spatial import cKDTree

# ----------------------------- small helpers -----------------------------
def _find_col(df: pd.DataFrame, candidates):
    """
    Return the first matching column name in `df` from a list of `candidates`,
    case-insensitive. If none found, return None.
    """
    lowermap = {c.lower(): c for c in df.columns}
    for name in candidates:
        if name is None:
            continue
        if name.lower() in lowermap:
            return lowermap[name.lower()]
    return None

def _find_col_contains(df: pd.DataFrame, fragments):
    """
    Return the first column in `df` whose lowercase name contains ANY of the
    strings in `fragments` (list of substrings). If none found, return None.
    """
    cols_low = [c.lower() for c in df.columns]
    for frag in fragments:
        if frag is None:
            continue
        frag = frag.lower()
        for c_low, c in zip(cols_low, df.columns):
            if frag in c_low:
                return c
    return None

def _decimal_year_to_timestamp(dy):
    """
    Convert decimal year(s) to UTC pandas Timestamp(s).
    Works with a scalar float or an array-like of floats.

    Method: linear interpolation between Jan-01 of year and Jan-01 of next year.
    """
    dy = np.asarray(dy, dtype=float)
    year = np.floor(dy).astype(int)
    frac = dy - year
    starts = pd.to_datetime([f"{y}-01-01T00:00:00Z" for y in year], utc=True)
    ends   = pd.to_datetime([f"{y+1}-01-01T00:00:00Z" for y in year], utc=True)
    dt = starts + (ends - starts) * frac
    return dt if dt.size > 1 else dt[0]

def _parse_time_col(df: pd.DataFrame) -> pd.Series:
    """
    Generic time parser for IGRF-like CSVs.

    Tries, in order:
    1) ISO-like timestamps in a 'time'/'date' column.
    2) Decimal year columns (decimalyear, decyear, etc).
    3) Split 'Date' + 'Time' columns if present.
    4) Y/M/D/H/M/S component columns (year, month, day, hour, minute, second)
       and day-of-year (doy) fallback.

    Returns a tz-aware (UTC) Series; raises ValueError if parsing fails.
    """
    # 1) direct time/date column
    tcol = _find_col(df, ["time", "datetime", "date", "DATE", "Date"])
    if tcol is not None:
        t = pd.to_datetime(df[tcol], errors="coerce", utc=True)
        if t.notna().any():
            return t

    # 2) decimal year
    dycol = _find_col(df, ["decimalyear","decimal_year","decyear","year_decimal","year_dec"])
    if dycol is not None:
        dy = pd.to_numeric(df[dycol], errors="coerce")
        t = pd.to_datetime(_decimal_year_to_timestamp(dy), utc=True)
        if t.notna().any():
            return t

    # 3) split date + time
    dcol = _find_col(df, ["date"])
    ctime = _find_col(df, ["time"])
    if dcol is not None and ctime is not None:
        tt = pd.to_datetime(df[dcol].astype(str) + " " + df[ctime].astype(str), errors="coerce", utc=True)
        if tt.notna().any():
            return tt

    # 4) Y/M/D/H/M/S or Y + DOY
    ycol = _find_col_contains(df, ["year"])
    mcol = _find_col_contains(df, ["month"])
    dday = _find_col_contains(df, ["day"])
    hcol = _find_col_contains(df, ["hour"])
    minc = _find_col_contains(df, ["minute","min"])
    scol = _find_col_contains(df, ["second","sec"])
    doy  = _find_col_contains(df, ["doy","dayofyear","julian"])

    if ycol is not None and doy is not None:
        year = pd.to_numeric(df[ycol], errors="coerce").astype("Int64")
        day  = pd.to_numeric(df[doy], errors="coerce")
        # build Jan-01 + (doy-1) days
        base = pd.to_datetime([f"{int(y)}-01-01" if pd.notna(y) else "NaT" for y in year], utc=True, errors="coerce")
        t = base + pd.to_timedelta(day - 1, unit="D")
        # add optional hour/min/sec
        if hcol is not None:
            t = t + pd.to_timedelta(pd.to_numeric(df[hcol], errors="coerce").fillna(0), unit="h")
        if minc is not None:
            t = t + pd.to_timedelta(pd.to_numeric(df[minc], errors="coerce").fillna(0), unit="m")
        if scol is not None:
            t = t + pd.to_timedelta(pd.to_numeric(df[scol], errors="coerce").fillna(0), unit="s")
        if t.notna().any():
            return t

    if ycol is not None and mcol is not None and dday is not None:
        parts = (
            pd.to_numeric(df[ycol], errors="coerce").astype("Int64").astype(str) + "-" +
            pd.to_numeric(df[mcol], errors="coerce").astype("Int64").astype(str) + "-" +
            pd.to_numeric(df[dday], errors="coerce").astype("Int64").astype(str)
        )
        if hcol is not None:
            parts = parts + " " + pd.to_numeric(df[hcol], errors="coerce").fillna(0).astype(int).astype(str)
        if minc is not None:
            parts = parts + ":" + pd.to_numeric(df[minc], errors="coerce").fillna(0).astype(int).astype(str)
        if scol is not None:
            parts = parts + ":" + pd.to_numeric(df[scol], errors="coerce").fillna(0).astype(int).astype(str)
        t = pd.to_datetime(parts, errors="coerce", utc=True)
        if t.notna().any():
            return t

    raise ValueError("Could not parse time in IGRF CSV. Tried ISO, decimal year, Y/M/D, and Y+DOY.")

def _parse_lat_lon(df: pd.DataFrame):
    """
    Return (lat, lon) Series from common column names.
    """
    latc = _find_col(df, ["lat","latitude","Lat","Latitude"])
    lonc = _find_col(df, ["lon","longitude","Lon","Longitude"])
    if latc is None or lonc is None:
        raise ValueError("Could not find latitude/longitude columns in IGRF CSV.")
    lat = pd.to_numeric(df[latc], errors="coerce")
    lon = pd.to_numeric(df[lonc], errors="coerce")
    return lat, lon

def _choose_value_column(df: pd.DataFrame, comp_name: str, to_drop: set | None = None):
    """
    Heuristic to pick the column with component values (X, Y, Z, F, Dec, Inc).
    - Prefers a column whose name matches comp_name (regex, case-insensitive).
    - Else, picks the last numeric column not in `to_drop`.
    Returns column name or None.
    """
    if to_drop is None:
        to_drop = set()
    rx = re.compile(comp_name, re.IGNORECASE)
    cand = [c for c in df.columns if c not in to_drop]
    for c in cand:
        if rx.search(str(c)):
            return c
    num_cand = [c for c in cand if pd.api.types.is_numeric_dtype(df[c])]
    if num_cand:
        return num_cand[-1]
    return cand[-1] if cand else None

# ----------------------------- IGRF CSV readers -----------------------------
def _read_igrf_component_csv(path, comp_name):
    """
    Return standardized columns: ['time','lat','lon', comp_name].

    Supports:
    - NCEI 'igrfgridData_*.csv' (header lines start with '#'; numeric body with columns:
      DecimalYear, Latitude, Longitude, Elevation_km, Value, SecVar)
    - Generic CSVs with flexible time parsing.

    Parameters
    ----------
    path : str | Path
        CSV file path.
    comp_name : {"X","Y","Z","F","Dec","Inc"}
        The scalar component name to extract/assign for the 'Value' column.

    Notes
    -----
    - Longitudes are normalized to [-180, 180].
    - Time is UTC tz-aware.
    """
    p = Path(path)

    # --- Fast path: NCEI grid format (commented header + fixed 6 numeric columns) ---
    try:
        with open(p, "r", encoding="utf-8") as f:
            first = ""
            for line in f:
                if line.strip():
                    first = line.strip()
                    break
        if first.startswith("#") or first.startswith("############################################################"):
            df = pd.read_csv(
                p,
                comment="#",
                header=None,
                names=["DecimalYear", "Latitude", "Longitude", "Elevation_km", "Value", "SecVar"],
            )
            df = df.dropna(subset=["DecimalYear", "Latitude", "Longitude", "Value"])
            t   = pd.to_datetime(_decimal_year_to_timestamp(pd.to_numeric(df["DecimalYear"], errors="coerce")), utc=True)
            lat = pd.to_numeric(df["Latitude"], errors="coerce")
            lon = pd.to_numeric(df["Longitude"], errors="coerce")
            val = pd.to_numeric(df["Value"], errors="coerce")
            out = pd.DataFrame({"time": t, "lat": lat, "lon": lon, comp_name: val})
            out = out.dropna(subset=["time", "lat", "lon"])
            out["lon"] = ((out["lon"] + 180) % 360) - 180
            return out
    except Exception:
        # fall back to generic parser if the quick path fails
        pass

    # --- Generic flexible parser (headered files) ---
    df = pd.read_csv(p, sep=",", on_bad_lines="skip")
    t = _parse_time_col(df)
    lat, lon = _parse_lat_lon(df)
    drop = set([
        _find_col(df, ["time","datetime","date","DATE","Date"]),
        _find_col(df, ["decimalyear","decimal_year","decyear","year_decimal"]),
        _find_col_contains(df, ["date","time","epoch","timestamp","yr"]),
        _find_col(df, ["lat","latitude","Lat","Latitude"]),
        _find_col_contains(df, ["lat"]),
        _find_col(df, ["lon","longitude","Lon","Longitude"]),
        _find_col_contains(df, ["lon","long"]),
        _find_col_contains(df, ["year","month","day","doy","dayofyear","julian"]),
    ])
    drop = {c for c in drop if c is not None}
    valcol = _choose_value_column(df, comp_name, drop)
    if valcol is None:
        raise ValueError(f"Could not identify value column for {comp_name} in {p.name}. Columns: {list(df.columns)}")
    val = pd.to_numeric(df[valcol], errors="coerce")
    out = pd.DataFrame({"time": t, "lat": lat, "lon": lon, comp_name: val})
    out = out.dropna(subset=["time","lat","lon"])
    out["lon"] = ((out["lon"] + 180) % 360) - 180
    return out

def load_igrf_folder(folder):
    """
    Load IGRF CSVs from a folder and return a single wide table with columns:
    ['time','lat','lon','X','Y','Z','F','Dec','Inc']  (subset if some components missing).

    File detection:
    - Tries to infer the component (X,Y,Z,F,Dec,Inc) from the filename, e.g.
      'igrfgridData_x.csv', '..._dec.csv', '..._inc.csv', etc.
    - Falls back to substring guesses for 'decl', 'inc', 'dip', and 'igrf' + {x,y,z,f}.

    Parameters
    ----------
    folder : str | Path
        Folder containing *.csv IGRF component files.

    Returns
    -------
    DataFrame
        Longitudes normalized to [-180,180], time in UTC, outer-merged on ['time','lat','lon'].
    """
    folder = Path(folder)
    if not folder.exists():
        raise FileNotFoundError(f"Folder not found: {folder}")

    comp_map = {"x": "X", "y": "Y", "z": "Z", "f": "F", "dec": "Dec", "incl": "Inc", "inc": "Inc"}
    pieces = []
    for p in folder.glob("*.csv"):
        name = p.name.lower()
        comp = None
        for k, v in comp_map.items():
            if re.search(rf"(^|[_-]){k}([_.-]|$)", name):
                comp = v
                break
        if comp is None:
            if "decl" in name or "dec" in name: comp = "Dec"
            elif "inc" in name or "incl" in name or "dip" in name: comp = "Inc"
            elif "x" in name and "igrf" in name: comp = "X"
            elif "y" in name and "igrf" in name: comp = "Y"
            elif "z" in name and "igrf" in name: comp = "Z"
            elif "f" in name and "igrf" in name: comp = "F"
        if comp is None:
            continue
        try:
            piece = _read_igrf_component_csv(p, comp)
            pieces.append(piece)
        except Exception as e:
            print(f"[WARN] {p.name}: {e}")

    if not pieces:
        raise RuntimeError(f"No recognizable IGRF CSVs found in {folder}")

    # Outer-merge all pieces on common keys
    df = pieces[0]
    for pc in pieces[1:]:
        df = df.merge(pc, on=["time","lat","lon"], how="outer")

    df = df.drop_duplicates(subset=["time","lat","lon"]).sort_values(["time","lat","lon"])
    keep_cols = ["time","lat","lon","X","Y","Z","F","Dec","Inc"]
    df = df[[c for c in keep_cols if c in df.columns]].copy()
    return df

# ----------------------------- survey-time helpers -----------------------------
def _ensure_survey_time(df, date_col="Reading_Date", time_col="Reading_Time"):
    """
    Return a tz-aware UTC pandas Series for survey timestamps.
    Prefers existing '__survey_datetime_utc' column if present.
    """
    if "__survey_datetime_utc" in df.columns:
        return pd.to_datetime(df["__survey_datetime_utc"], errors="coerce", utc=True)
    return pd.to_datetime(
        df[date_col].astype(str) + " " + df[time_col].astype(str),
        format="%d-%b-%Y %H:%M:%S.%f", errors="coerce", utc=True
    )

def _nearest_time_index(survey_time_utc, grid_times_utc, tolerance="365D"):
    """
    For each survey time, return the integer index of the *nearest* grid time,
    or -1 when the nearest grid time is outside the provided tolerance.

    Parameters
    ----------
    survey_time_utc : array-like of datetime64[ns, UTC] or strings
    grid_times_utc  : array-like of datetime64[ns, UTC] or strings
    tolerance       : str or Timedelta (e.g., "365D")

    Returns
    -------
    numpy.ndarray[int]
        Indices into the **sorted unique** grid times array (length = len(survey_time_utc)).
    """
    s = pd.DataFrame({"i": np.arange(len(survey_time_utc)),
                      "time": pd.to_datetime(survey_time_utc, utc=True, errors="coerce")})
    g = pd.DataFrame({"time": pd.Index(pd.to_datetime(grid_times_utc, utc=True)).sort_values().unique()})\
            .reset_index().rename(columns={"index":"g_idx"})
    out = np.full(len(s), -1, dtype=int)
    s_valid = s[s["time"].notna()].sort_values("time")
    if s_valid.empty or g.empty:
        return out
    joined = pd.merge_asof(s_valid, g, on="time", direction="nearest",
                           tolerance=pd.Timedelta(tolerance))
    gid = joined["g_idx"].to_numpy()
    ok = np.isfinite(gid)
    out[s_valid.loc[ok, "i"].to_numpy()] = gid[ok].astype(int)
    return out

# ----------------------------- 1) attach IGRF to survey -----------------------------
def attach_igrf_to_survey(
    df_survey: pd.DataFrame,
    df_igrf: pd.DataFrame,
    lat_col: str = "Mag_Latitude",
    lon_col: str = "Mag_Longitude",
    date_col: str = "Reading_Date",
    time_col: str = "Reading_Time",
    time_tolerance: str = "365D"
) -> pd.DataFrame:
    """
    Attach IGRF main-field *direction* and components to each survey row by:
      (a) matching the nearest IGRF time (within `time_tolerance`) and
      (b) taking the nearest spatial neighbor (KDTree) within that time slice.

    Adds columns:
      - 'IGRF_time' (UTC)
      - 'IGRF_X','IGRF_Y','IGRF_Z' (nT) when available or reconstructed from F/Dec/Inc
      - 'IGRF_F','IGRF_Dec','IGRF_Inc' when available
      - 'IGRF_bx','IGRF_by','IGRF_bz' (unit vector of main field at nearest node)
      - 'IGRF_nn_dist_deg' (great-circle approx. in degrees using lat/lon Euclidean)

    Parameters
    ----------
    df_survey : DataFrame
        Survey table containing lat/lon (WGS84) and timestamps.
    df_igrf : DataFrame
        IGRF grid table from `load_igrf_folder`, with columns 'time','lat','lon' and
        any subset of 'X','Y','Z','F','Dec','Inc'.
    lat_col, lon_col : str
        Names of survey latitude/longitude columns (degrees).
    date_col, time_col : str
        Names of survey date/time columns if '__survey_datetime_utc' is absent.
    time_tolerance : str
        Max time separation for nearest-match, e.g., "365D" for a year.

    Returns
    -------
    DataFrame
        Copy of `df_survey` with the columns listed above.
    """
    out = df_survey.copy()

    # timestamps
    dt = _ensure_survey_time(out, date_col, time_col)
    out["__dt_utc__"] = dt

    # unique grid times
    grid_times = pd.Index(pd.to_datetime(df_igrf["time"].dropna().unique(), utc=True)).sort_values()
    if len(grid_times) == 0:
        raise RuntimeError("IGRF grid has no time values.")

    # map each survey row to nearest grid time index (or -1)
    t_idx = _nearest_time_index(dt, grid_times, tolerance=time_tolerance)

    # prepare output columns
    cols = ["IGRF_time","IGRF_X","IGRF_Y","IGRF_Z","IGRF_F","IGRF_Dec","IGRF_Inc","IGRF_bx","IGRF_by","IGRF_bz",
            "IGRF_nn_dist_deg"]
    for c in cols:
        out[c] = np.nan
    out["IGRF_time"] = pd.NaT

    # group survey rows by chosen grid time
    groups = {}
    for pos, gi in enumerate(t_idx):
        if gi >= 0:
            groups.setdefault(int(gi), []).append(pos)

    # process each time slice
    for gi, rows in groups.items():
        t_sel = pd.to_datetime(grid_times[gi])

        gslice = df_igrf[df_igrf["time"] == t_sel]
        if gslice.empty:
            continue

        # finite grid coords
        glat = pd.to_numeric(gslice["lat"], errors="coerce").to_numpy()
        glon = pd.to_numeric(gslice["lon"], errors="coerce").to_numpy()
        gmask = np.isfinite(glat) & np.isfinite(glon)
        if not np.any(gmask):
            continue

        G = np.column_stack([glat[gmask], glon[gmask]])
        tree = cKDTree(G)

        # survey coords for these rows
        qlat_all = pd.to_numeric(out.iloc[rows][lat_col], errors="coerce").to_numpy()
        qlon_all = pd.to_numeric(out.iloc[rows][lon_col], errors="coerce").to_numpy()
        qmask = np.isfinite(qlat_all) & np.isfinite(qlon_all)
        if not np.any(qmask):
            continue

        Q = np.column_stack([qlat_all[qmask], qlon_all[qmask]])
        dist, j = tree.query(Q, k=1, workers=-1)

        # helper to fetch component arrays aligned with gmask
        def _get(name):
            return pd.to_numeric(gslice[name], errors="coerce").to_numpy()[gmask] if name in gslice.columns else None

        Xg, Yg, Zg = _get("X"), _get("Y"), _get("Z")
        Fg, Dg, Ig = _get("F"), _get("Dec"), _get("Inc")

        # reconstruct X/Y/Z if missing
        if Xg is None or Yg is None or Zg is None:
            if Fg is None or Dg is None or Ig is None:
                # no usable field info in this slice
                continue
            Dd = np.deg2rad(Dg); Id = np.deg2rad(Ig)
            Xg = Fg*np.cos(Id)*np.cos(Dd)
            Yg = Fg*np.cos(Id)*np.sin(Dd)
            Zg = Fg*np.sin(Id)

        # nearest values (vector length -> unit vector)
        Xq, Yq, Zq = Xg[j], Yg[j], Zg[j]
        B = np.sqrt(Xq*Xq + Yq*Yq + Zq*Zq)
        ok = np.isfinite(B) & (B > 0)

        bx = np.full_like(B, np.nan, dtype=float)
        by = np.full_like(B, np.nan, dtype=float)
        bz = np.full_like(B, np.nan, dtype=float)
        bx[ok] = Xq[ok]/B[ok]; by[ok] = Yq[ok]/B[ok]; bz[ok] = Zq[ok]/B[ok]

        # write back only to rows with finite survey coords
        sel = np.asarray(rows, dtype=int)[qmask]
        # safer assignment by column labels
        out.loc[sel, "IGRF_time"] = pd.to_datetime([t_sel]*len(sel), utc=True)
        out.loc[sel, "IGRF_X"]    = Xq
        out.loc[sel, "IGRF_Y"]    = Yq
        out.loc[sel, "IGRF_Z"]    = Zq
        if "F" in df_igrf.columns:
            out.loc[sel, "IGRF_F"] = Fg[j]
        if "Dec" in df_igrf.columns:
            out.loc[sel, "IGRF_Dec"] = Dg[j]
        if "Inc" in df_igrf.columns:
            out.loc[sel, "IGRF_Inc"] = Ig[j]
        out.loc[sel, "IGRF_bx"]  = bx
        out.loc[sel, "IGRF_by"]  = by
        out.loc[sel, "IGRF_bz"]  = bz
        out.loc[sel, "IGRF_nn_dist_deg"] = dist

    return out

# ----------------------------- 2) projection + correction -----------------------------
def apply_projection_and_correction(
    df_with_igrf: pd.DataFrame,
    x_col: str = "VSSX_vss", y_col: str = "VSSY_vss", z_col: str = "VSSZ_vss",
    field_col: str = "Magnetic_Field_despiked",
    baseline: str = "median"
) -> pd.DataFrame:
    """
    Project observatory disturbances (dX,dY,dZ) onto the local main-field direction
    defined by attached IGRF unit vector (bx,by,bz), then subtract from the field.

    Steps:
      1) Build a UTC DatetimeIndex from '__dt_utc__'.
      2) For each day (UTC), compute a baseline (median/mean) for X/Y/Z and subtract
         to obtain disturbances dX,dY,dZ.
      3) Compute projection:  diurnal_proj_nT = dX*bx + dY*by + dZ*bz
      4) Correct the selected field: field_col + '_corr' = field - diurnal_proj_nT.

    Parameters
    ----------
    df_with_igrf : DataFrame
        Survey table AFTER `attach_igrf_to_survey`, containing:
        '__dt_utc__', 'IGRF_bx','IGRF_by','IGRF_bz', and the observatory X/Y/Z.
    x_col, y_col, z_col : str
        Column names for observatory XYZ (nT), per-sample (time-aligned with survey).
    field_col : str
        Name of the magnetic field column to be corrected (e.g., despiked scalar).
    baseline : {"median","mean"}
        Daily baseline estimator for XYZ (robust 'median' recommended).

    Returns
    -------
    DataFrame
        Copy with new columns:
          - 'diurnal_proj_nT' (projection of dX/Y/Z onto bx/by/bz)
          - f'{field_col}_corr' (corrected field)
    """
    out = df_with_igrf.copy()
    dt = pd.to_datetime(out["__dt_utc__"], errors="coerce", utc=True)

    # clean observatory XYZ (strip common sentinels)
    Xo = pd.to_numeric(out.get(x_col), errors="coerce").replace({88888.0: np.nan, 99999.0: np.nan})
    Yo = pd.to_numeric(out.get(y_col), errors="coerce").replace({88888.0: np.nan, 99999.0: np.nan})
    Zo = pd.to_numeric(out.get(z_col), errors="coerce").replace({88888.0: np.nan, 99999.0: np.nan})

    # DatetimeIndex for day-wise baselines; keep positional alignment
    idx = pd.DatetimeIndex(dt).tz_convert("UTC")

    def _disturbance(v: pd.Series) -> pd.Series:
        """
        Compute v - daily_baseline(v) with the chosen estimator.
        Uses the same positional order as input and returns a Series indexed by `idx`.
        """
        s = pd.Series(v.values, index=idx)
        day = s.index.normalize()
        if baseline == "median":
            base = s.groupby(day).transform("median")
        else:
            base = s.groupby(day).transform("mean")
        return s - base

    dX = _disturbance(Xo)
    dY = _disturbance(Yo)
    dZ = _disturbance(Zo)

    # IGRF unit vector components
    bx = pd.to_numeric(out["IGRF_bx"], errors="coerce")
    by = pd.to_numeric(out["IGRF_by"], errors="coerce")
    bz = pd.to_numeric(out["IGRF_bz"], errors="coerce")

    # boolean mask aligned by position, using NumPy arrays to avoid index alignment issues
    ok = (
        dt.notna().to_numpy()
        & np.isfinite(dX.to_numpy())
        & np.isfinite(dY.to_numpy())
        & np.isfinite(dZ.to_numpy())
        & np.isfinite(bx.to_numpy())
        & np.isfinite(by.to_numpy())
        & np.isfinite(bz.to_numpy())
    )

    # projection
    proj = np.full(len(out), np.nan, dtype=float)
    proj[ok] = (
        dX.to_numpy()[ok] * bx.to_numpy()[ok]
        + dY.to_numpy()[ok] * by.to_numpy()[ok]
        + dZ.to_numpy()[ok] * bz.to_numpy()[ok]
    )

    out["diurnal_proj_nT"] = proj
    out[f"{field_col}_corr"] = pd.to_numeric(out[field_col], errors="coerce") - out["diurnal_proj_nT"]

    # quick diagnostics for sanity
    print(f"Attached IGRF on {out['IGRF_time'].notna().sum()} / {len(out)} rows.")
    print(f"Finite bx/by/bz on {np.isfinite(bx.to_numpy()).sum()} / {len(out)} rows.")
    print(f"Projected rows: {np.isfinite(proj).sum()} / {len(out)}.")

    return out

In [17]:
# 1) Read your IGRF grid folder (files like igrfgridData_x.csv, ..._y.csv, ..._dec.csv, etc.)
IGRF_DIR = "marco-mag/data/igrf"
df_igrf = load_igrf_folder(IGRF_DIR)

df_igrf.head()

,time,lat,lon,X,Y,Z,F,Dec,Inc
0,2025-08-19 00:01:35.039999464+00:00,-25.07762,-48.0,15990.3,-6131.3,-14775.4,22618.5,-20.97884,-40.78664
1,2025-08-19 00:01:35.039999464+00:00,-25.07762,-47.9,15970.9,-6137.3,-14805.4,22626.0,-21.02064,-40.87054
2,2025-08-19 00:01:35.039999464+00:00,-25.07762,-47.8,15951.6,-6143.1,-14835.4,22633.6,-21.06214,-40.95438
3,2025-08-19 00:01:35.039999464+00:00,-25.07762,-47.7,15932.3,-6148.8,-14865.3,22641.2,-21.10333,-41.03817
4,2025-08-19 00:01:35.039999464+00:00,-25.07762,-47.6,15912.9,-6154.4,-14895.3,22648.8,-21.14421,-41.12189


In [18]:
# 2) Attach IGRF to each survey row (your df_igrf already built from igrfgridData_*.csv)
df_joined = attach_igrf_to_survey(
    df_poly, df_igrf,
    lat_col="Mag_Latitude", lon_col="Mag_Longitude",
    date_col="Reading_Date", time_col="Reading_Time",
    time_tolerance="365D"
)

/tmp/ipykernel_29802/3549638115.py:473: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'DatetimeIndex(['2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               ...
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
               '2025-08-19 00:01:35.039999464+00:00',
  

In [19]:
# 3) Projection + correction
df_corr = apply_projection_and_correction(
    df_joined,
    x_col="VSSX_vss", y_col="VSSY_vss", z_col="VSSZ_vss",
    field_col="Magnetic_Field_poly",
    baseline="median"
)

Attached IGRF on 78377 / 78418 rows.
Finite bx/by/bz on 78377 / 78418 rows.
Projected rows: 78377 / 78418.


In [20]:
# df_corr.isna().sum()

In [21]:
# df_corr.head()

In [22]:
df_corr

,Reading_Date,Reading_Time,Magnetic_Field,Signal_Quality,Depth,Leak,Measurement_Time,Signal_Strength,Gradient_Condition,Weak_Signal,...,IGRF_Z,IGRF_F,IGRF_Dec,IGRF_Inc,IGRF_bx,IGRF_by,IGRF_bz,IGRF_nn_dist_deg,diurnal_proj_nT,Magnetic_Field_poly_corr
0,20-Aug-2025,17:36:36.700,22759.127,79,0.0,0,215,119,NaN,NaN,...,-14775.4,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039767,-18.566091,22777.693091
1,20-Aug-2025,17:36:37.200,22757.742,79,0.0,0,215,122,NaN,NaN,...,-14775.4,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039767,-18.567389,22776.309389
2,20-Aug-2025,17:36:37.700,22756.970,89,0.0,0,215,129,NaN,NaN,...,-14775.4,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039769,-18.568686,22775.538686
3,20-Aug-2025,17:36:38.700,22753.489,89,0.0,0,215,133,NaN,NaN,...,-14775.4,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039771,-18.571282,22771.516580
4,20-Aug-2025,17:36:39.200,22750.916,89,0.0,0,215,134,NaN,NaN,...,-14775.4,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039772,-18.572579,22769.588638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78413,21-Aug-2025,18:00:44.700,22585.304,89,0.0,0,215,132,NaN,NaN,...,-14781.2,22630.2,-21.04502,-40.78059,0.706711,-0.271916,-0.653163,0.036347,-21.982823,22607.286823
78414,21-Aug-2025,18:00:45.200,22585.829,89,0.0,0,215,129,NaN,NaN,...,-14781.2,22630.2,-21.04502,-40.78059,0.706711,-0.271916,-0.653163,0.036349,-21.988607,22607.817607
78415,21-Aug-2025,18:00:45.700,22585.503,89,0.0,0,215,133,NaN,NaN,...,-14781.2,22630.2,-21.04502,-40.78059,0.706711,-0.271916,-0.653163,0.036350,-21.994390,22607.497390
78416,21-Aug-2025,18:00:46.200,22585.352,89,0.0,0,215,133,NaN,NaN,...,-14781.2,22630.2,-21.04502,-40.78059,0.706711,-0.271916,-0.653163,0.036352,-22.000173,22607.352173


### Visualizing the time-series

In [23]:
make_line_browser(df_corr,
                  line_col="Line_Name",
                  date_col="Reading_Date", time_col="Reading_Time",
                  lat_col="Mag_Northing", lon_col="Mag_Easting",
                  survey_col="Magnetic_Field_poly_corr",
                  geomag_col="Magnetic_Field_poly",
                  distance_col="Profile_Dist_m"
                  )

## Removing the IGRF-14 from the survey data

In [24]:
## Subtract it from the corrected data
df_corr['AMF'] = df_corr['Magnetic_Field_poly_corr'] - df_corr['IGRF_F']

In [25]:
df_corr.head()

,Reading_Date,Reading_Time,Magnetic_Field,Signal_Quality,Depth,Leak,Measurement_Time,Signal_Strength,Gradient_Condition,Weak_Signal,...,IGRF_F,IGRF_Dec,IGRF_Inc,IGRF_bx,IGRF_by,IGRF_bz,IGRF_nn_dist_deg,diurnal_proj_nT,Magnetic_Field_poly_corr,AMF
0,20-Aug-2025,17:36:36.700,22759.127,79,0.0,0,215,119,NaN,NaN,...,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039767,-18.566091,22777.693091,159.193091
1,20-Aug-2025,17:36:37.200,22757.742,79,0.0,0,215,122,NaN,NaN,...,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039767,-18.567389,22776.309389,157.809389
2,20-Aug-2025,17:36:37.700,22756.970,89,0.0,0,215,129,NaN,NaN,...,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039769,-18.568686,22775.538686,157.038686
3,20-Aug-2025,17:36:38.700,22753.489,89,0.0,0,215,133,NaN,NaN,...,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039771,-18.571282,22771.516580,153.016580
4,20-Aug-2025,17:36:39.200,22750.916,89,0.0,0,215,134,NaN,NaN,...,22618.5,-20.97884,-40.78664,0.706958,-0.271075,-0.653245,0.039772,-18.572579,22769.588638,151.088638


In [26]:
make_line_browser(df_corr,
                  line_col="Line_Name",
                  date_col="Reading_Date", time_col="Reading_Time",
                  lat_col="Mag_Northing", lon_col="Mag_Easting",
                  survey_col="AMF",
                  geomag_col=None,
                  distance_col="Profile_Dist_m"
                  )

### Exporting the corrected data

In [27]:
data_dir

'marco-mag/data/Magnetometry/merged_data'

In [28]:
# --- CÉLULA DE CHECAGEM (Adicionar após a [28]) ---

# Garante que estamos olhando apenas para dados com coordenadas válidas
df_check = df_corr.dropna(subset=['Mag_Easting', 'Mag_Northing', 'Mag_Latitude', 'Mag_Longitude'])

print("--- Checagem de Coordenadas (Metros UTM) ---")
min_easting = df_check.Mag_Easting.min()
max_easting = df_check.Mag_Easting.max()
min_northing = df_check.Mag_Northing.min()
max_northing = df_check.Mag_Northing.max()

print(f"Easting (X) Mín: {min_easting:.2f} m")
print(f"Easting (X) Máx: {max_easting:.2f} m")
print(f"Largura (X): {(max_easting - min_easting) / 1000:.2f} km")
print("")
print(f"Northing (Y) Mín: {min_northing:.2f} m")
print(f"Northing (Y) Máx: {max_northing:.2f} m")
print(f"Altura (Y): {(max_northing - min_northing) / 1000:.2f} km")
print("\n" + "="*40 + "\n")

print("--- Checagem de Coordenadas (Graus Decimais) ---")
min_lon = df_check.Mag_Longitude.min()
max_lon = df_check.Mag_Longitude.max()
min_lat = df_check.Mag_Latitude.min()
max_lat = df_check.Mag_Latitude.max()

print(f"Longitude Mínima: {min_lon:.6f}°")
print(f"Longitude Máxima: {max_lon:.6f}°")
print(f"Latitude Mínima: {min_lat:.6f}°")
print(f"Latitude Máxima: {max_lat:.6f}°")

--- Checagem de Coordenadas (Metros UTM) ---
Easting (X) Mín: 197246.83 m
Easting (X) Máx: 802754.12 m
Largura (X): 605.51 km

Northing (Y) Mín: 7223106.95 m
Northing (Y) Máx: 7238185.30 m
Altura (Y): 15.08 km


--- Checagem de Coordenadas (Graus Decimais) ---
Longitude Mínima: -48.028437°
Longitude Máxima: -47.908510°
Latitude Mínima: -25.077668°
Latitude Máxima: -24.943241°


In [30]:
# --- CÉLULA [28b] - O PATCH DE CORREÇÃO ---

print("--- Aplicando o patch de coordenadas ---")
print(f"Max Lon ANTES da correção (em df_corr): {df_corr['Mag_Longitude'].max()}")

# Como df_raw e df_corr têm o mesmo índice, podemos copiar as colunas "boas"
# do df_raw (da Célula 1) para o df_corr (final).

cols_para_corrigir = [
    'Mag_Latitude',
    'Mag_Longitude',
    'Mag_Easting',
    'Mag_Northing'
]

# Força as colunas em df_corr a serem idênticas às colunas em df_raw
df_corr[cols_para_corrigir] = df_raw[cols_para_corrigir]

print(f"Max Lon DEPOIS da correção (em df_corr): {df_corr['Mag_Longitude'].max()}")
print("Patch aplicado. As coordenadas agora estão corretas.")

--- Aplicando o patch de coordenadas ---
Max Lon ANTES da correção (em df_corr): -47.90851044
Max Lon DEPOIS da correção (em df_corr): -47.90851044
Patch aplicado. As coordenadas agora estão corretas.


In [29]:
df_corr.to_csv(f'{data_dir}/can_mag_corr.csv', index=False)